In [ ]:
def calc(file):
    import pandas as pd
    import numpy as np
    from numpy.lib.stride_tricks import sliding_window_view
    from os.path import isfile
    
    def getParams (fs, alphaMax=1000, df=100, fMin=5e+4):
        # alphaMax - max modulation frequency
        # df - carrier frequency resolution
        # fMin - min carrier frequency

        # STFT windows' hop
        R = int(np.floor(fs / (2 * alphaMax))) #shift of the stft
        # STFT window length
        Nw = int(fs / df)
        # number of STFT windows
    #     M = int((x.size - Nw) / R + 1)

        # hannind window
        w = np.hanning(Nw)
        # Dirichlet kernel parameter
        P = int(np.round((Nw - 1) / (2 * R)))
        # Dirichlet kernel
        D = np.sum(
            [np.exp(2 * np.pi * 1j * p *(np.arange(Nw) - Nw / 2) / Nw) for p in np.arange(- P, P + 1)], 
            axis=0
        )
        D = D.real
        return fMin, R, Nw, w, D

    def getCS(x, fs):
        fMin, R, Nw, w, D = getParams(fs)

        _, _, X_w = stft(x, fs=fs, window=w, nperseg=Nw, noverlap=Nw - R, nfft=Nw, return_onesided=True)
        f, t, x_w_d = stft(x, fs=fs, window=w * D, nperseg=Nw, noverlap=Nw - R, nfft=Nw, return_onesided=True)

        # here I save some computation time (factor of fMin / fs ~ 4) but later I have to implement the approsimation 
        # to the coherence by neglecting the frequency shift CCoh = CS / CS[0, :]
        X_w = X_w[f >= fMin, :]
        x_w_d = x_w_d[f >= fMin, :]

        CS = np.fft.fft(np.conjugate(X_w) * x_w_d, axis=1).T

        alpha = np.fft.fftfreq(x_w_d.shape[1], R / fs)
        CS = CS[alpha >= 0, :]
        CCoh = CS / CS[0, :]

        alpha = alpha[alpha >= 0]
        f = f[f >= fMin]
        
        EES_CS = np.sum(np.abs(CS), axis=1)
        EES_CCoh = np.sum(np.abs(CCoh), axis=1)

        alpha = alpha[alpha >= 0]
        f = f[f >= fMin]

        return CS, CCoh, f, alpha, EES_CS, EES_CCoh

    df = pd.read_feather(file)
    fs = 1 / df.iloc[1, 0] * 1000
    for iCh in range(1, df.shape[1]):
        newFileCS = file.replace('/', '/' + str(iCh) + '-cs-').replace('.fthr', '.npy')
        newFileCCoh = file.replace('/', '/' + str(iCh) + '-ccoh-').replace('.fthr', '.npy')
        newFileEES_CS = file.replace('/', '/' + str(iCh) + '-EEScs-').replace('.fthr', '.npy')
        newFileEES_CCoh = file.replace('/', '/' + str(iCh) + '-EESccoh-').replace('.fthr', '.npy')
        newFileAlpha = file.split('_16')[0] + '_alpha.npy'
        newFileFreq = file.split('_16')[0] + '_freq.npy'
        if isfile(newFileEES_CS) and isfile(newFileEES_CCoh):
            continue
        CS, CCoh, freq, alpha, EES_CS, EES_CCoh = getCS(df.iloc[:, iCh].to_numpy(), fs)
        
        if not isfile(newFileAlpha):
            np.save(newFileAlpha, alpha)
        if not isfile(newFileFreq):
            np.save(newFileFreq, freq)
#         np.save(newFileCS, CS)
#         np.save(newFileCCoh, CCoh)
#         np.save(newFileCS, CS)
#         np.save(newFileCCoh, CCoh)
        np.save(newFileEES_CS, EES_CS)
        np.save(newFileEES_CCoh, EES_CCoh)

In [ ]:
import ipyparallel as ipp
from glob import glob
from time import sleep


with ipp.Cluster() as rc:
    view = rc.load_balanced_view()
    files = sorted(glob("data/*.fthr"))
    asyncresult = view.map_async(calc, files)
    asyncresult.wait_interactive()
    asyncresult.get()

### Visualization

In [ ]:
files = glob("data/*.npy")
alphas = sorted([l for l in labels if 'alpha.npy' in l])
freqs = sorted([l for l in labels if 'freq.npy' in l])
EESccohS = [l for l in labels if 'EESccoh' in l]
EEScsS = [l for l in labels if 'EEScs' in l]

display(alphas)
display(freqs)
display(EESccohS)
display(EEScsS)

In [ ]:
from datetime import datetime

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

def plotSpectro(sensor, alpha, fileList):
    fileSensor = sorted([file for file in fileList if sensor in file])
    for fileType in fileSensor:
        print(fileType)
        files = sorted(glob('data/' + fileType + '*'))
        EESsNP = np.array([np.load(file) for file in files])
        time = [datetime.fromtimestamp(int(file.split('_')[-1].split('.')[0])) for file in files]
        alphaVec = np.load('data/' + alpha)
        vmax = np.percentile(EESsNP.T, 99.7)
        
        fig = px.imshow(
            EESsNP.T,
            labels=dict(y="Modulation Frequency [Hz]"),            
            x=time,
            y=alphaVec,
            title=fileType,
            zmax=vmax
               )
        
        fig.update_layout(
            yaxis = dict(
                tickmode = 'linear',
                dtick = 50,
                autorange=True,
            )
        )

        fig.write_html('visualization/'+ fileType + ".html")

for alpha in alphas:
    sensor = alpha.split('_alph')[0]
    plotSpectro(sensor, alpha, EESccohS)
    plotSpectro(sensor, alpha, EEScsS)